## Classificatori di cucina 2

In questa seconda lezione sulla classificazione, esploreremo `ulteriori modi` per classificare i dati categorici. Impareremo anche le implicazioni della scelta di un classificatore rispetto a un altro.

### [**Quiz pre-lezione**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **Prerequisiti**

Presumiamo che tu abbia completato le lezioni precedenti, poiché riprenderemo alcuni concetti già appresi.

Per questa lezione, avremo bisogno dei seguenti pacchetti:

-   `tidyverse`: Il [tidyverse](https://www.tidyverse.org/) è una [collezione di pacchetti R](https://www.tidyverse.org/packages) progettata per rendere la scienza dei dati più veloce, semplice e divertente!

-   `tidymodels`: Il framework [tidymodels](https://www.tidymodels.org/) è una [collezione di pacchetti](https://www.tidymodels.org/packages/) per la modellazione e il machine learning.

-   `themis`: Il pacchetto [themis](https://themis.tidymodels.org/) fornisce passaggi extra per le ricette utili a gestire dati sbilanciati.

Puoi installarli con il seguente comando:

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

In alternativa, lo script qui sotto verifica se hai i pacchetti necessari per completare questo modulo e li installa per te nel caso in cui manchino.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

## **1. Una mappa di classificazione**

Nella nostra [lezione precedente](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1), abbiamo cercato di rispondere alla domanda: come scegliere tra diversi modelli? In gran parte, dipende dalle caratteristiche dei dati e dal tipo di problema che vogliamo risolvere (ad esempio classificazione o regressione?).

In precedenza, abbiamo imparato le varie opzioni disponibili per classificare i dati utilizzando il cheat sheet di Microsoft. Il framework di Machine Learning di Python, Scikit-learn, offre un cheat sheet simile ma più dettagliato che può aiutare ulteriormente a restringere la scelta dei tuoi stimatori (un altro termine per classificatori):

<p >
   <img src="../../images/map.png"
   width="700"/>
   <figcaption></figcaption>


> Suggerimento: [visita questa mappa online](https://scikit-learn.org/stable/tutorial/machine_learning_map/) e segui il percorso per leggere la documentazione.
>
> Il [sito di riferimento di Tidymodels](https://www.tidymodels.org/find/parsnip/#models) fornisce anche un'ottima documentazione sui diversi tipi di modelli.

### **Il piano** 🗺️

Questa mappa è molto utile una volta che hai una chiara comprensione dei tuoi dati, poiché puoi "percorrere" i suoi sentieri per arrivare a una decisione:

-   Abbiamo \>50 campioni

-   Vogliamo prevedere una categoria

-   Abbiamo dati etichettati

-   Abbiamo meno di 100K campioni

-   ✨ Possiamo scegliere un Linear SVC

-   Se questo non funziona, dato che abbiamo dati numerici

    -   Possiamo provare un ✨ KNeighbors Classifier

        -   Se anche questo non funziona, provare ✨ SVC e ✨ Ensemble Classifiers

Questo è un percorso molto utile da seguire. Ora, entriamo subito nel vivo utilizzando il framework di modellazione [tidymodels](https://www.tidymodels.org/): una collezione coerente e flessibile di pacchetti R sviluppata per promuovere buone pratiche statistiche 😊.

## 2. Dividere i dati e gestire un dataset sbilanciato.

Dalle lezioni precedenti, abbiamo appreso che c'era un insieme di ingredienti comuni tra le nostre cucine. Inoltre, c'era una distribuzione piuttosto diseguale nel numero di cucine.

Affronteremo questi aspetti:

-   Eliminando gli ingredienti più comuni che creano confusione tra cucine distinte, utilizzando `dplyr::select()`.

-   Utilizzando una `recipe` che pre-elabora i dati per prepararli alla modellazione applicando un algoritmo di `over-sampling`.

Abbiamo già visto quanto sopra nella lezione precedente, quindi dovrebbe essere una passeggiata 🥳!


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### Gestire dati sbilanciati

I dati sbilanciati spesso hanno effetti negativi sulle prestazioni del modello. Molti modelli funzionano meglio quando il numero di osservazioni è uguale e, di conseguenza, tendono a incontrare difficoltà con dati non bilanciati.

Ci sono principalmente due modi per gestire set di dati sbilanciati:

-   aggiungere osservazioni alla classe minoritaria: `Over-sampling`, ad esempio utilizzando un algoritmo SMOTE che genera sinteticamente nuovi esempi della classe minoritaria basandosi sui vicini più prossimi di questi casi.

-   rimuovere osservazioni dalla classe maggioritaria: `Under-sampling`

Nella nostra lezione precedente, abbiamo dimostrato come gestire set di dati sbilanciati utilizzando una `recipe`. Una recipe può essere considerata come un piano che descrive quali passaggi devono essere applicati a un set di dati per prepararlo all'analisi. Nel nostro caso, vogliamo ottenere una distribuzione equa nel numero delle nostre cucine per il nostro `training set`. Passiamo subito all'azione.


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

Ora siamo pronti per addestrare i modelli 👩‍💻👨‍💻!

## 3. Oltre i modelli di regressione multinomiale

Nella lezione precedente, abbiamo esaminato i modelli di regressione multinomiale. Esploriamo ora alcuni modelli più flessibili per la classificazione.

### Support Vector Machines

Nel contesto della classificazione, i `Support Vector Machines` sono una tecnica di machine learning che cerca di trovare un *iperpiano* che separi "al meglio" le classi. Vediamo un esempio semplice:

<p >
   <img src="../../images/svm.png"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


H1~ non separa le classi. H2~ lo fa, ma solo con un piccolo margine. H3~ le separa con il margine massimo.

#### Classificatore Lineare a Vettori di Supporto

Il clustering a Vettori di Supporto (SVC) è un membro della famiglia delle macchine a vettori di supporto (SVM) nelle tecniche di apprendimento automatico. Nell'SVC, l'iperpiano viene scelto per separare correttamente `la maggior parte` delle osservazioni di addestramento, ma `potrebbe classificare erroneamente` alcune osservazioni. Consentendo ad alcuni punti di trovarsi sul lato sbagliato, l'SVM diventa più robusto agli outlier e quindi migliora la generalizzazione su nuovi dati. Il parametro che regola questa violazione è chiamato `cost` e ha un valore predefinito di 1 (vedi `help("svm_poly")`).

Creiamo un SVC lineare impostando `degree = 1` in un modello SVM polinomiale.


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

Ora che abbiamo catturato i passaggi di pre-elaborazione e la specifica del modello in un *workflow*, possiamo procedere con l'addestramento del linear SVC e valutare i risultati nel frattempo. Per quanto riguarda le metriche di performance, creiamo un set di metriche che valuterà: `accuracy`, `sensitivity`, `Positive Predicted Value` e `F Measure`.

> `augment()` aggiungerà colonna/e per le predizioni ai dati forniti.


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### Support Vector Machine

La support vector machine (SVM) è un'estensione del support vector classifier per gestire un confine non lineare tra le classi. In sostanza, le SVM utilizzano il *kernel trick* per ampliare lo spazio delle caratteristiche e adattarsi alle relazioni non lineari tra le classi. Una funzione kernel popolare e estremamente flessibile utilizzata dalle SVM è la *funzione di base radiale.* Vediamo come si comporterà sui nostri dati.


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Molto meglio 🤩!

> ✅ Si prega di consultare:
>
> -   [*Support Vector Machines*](https://bradleyboehmke.github.io/HOML/svm.html), Hands-on Machine Learning with R
>
> -   [*Support Vector Machines*](https://www.statlearning.com/), An Introduction to Statistical Learning with Applications in R
>
> per ulteriori approfondimenti.

### Classificatori Nearest Neighbor

Il metodo *K*-nearest neighbor (KNN) è un algoritmo in cui ogni osservazione viene predetta in base alla sua *somiglianza* con altre osservazioni.

Applichiamolo ai nostri dati.


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Sembra che questo modello non stia funzionando molto bene. Probabilmente modificare i parametri del modello (vedi `help("nearest_neighbor")`) migliorerà le prestazioni del modello. Assicurati di provarlo.

> ✅ Consulta:
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> per saperne di più sui classificatori *K*-Nearest Neighbors.

### Classificatori ensemble

Gli algoritmi ensemble funzionano combinando più stimatori di base per produrre un modello ottimale attraverso:

`bagging`: applicando una *funzione di media* a una collezione di modelli di base

`boosting`: costruendo una sequenza di modelli che si basano l'uno sull'altro per migliorare le prestazioni predittive.

Iniziamo provando un modello Random Forest, che costruisce una vasta collezione di alberi decisionali e poi applica una funzione di media per ottenere un modello complessivo migliore.


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Ottimo lavoro 👏!

Proviamo anche a sperimentare con un modello Boosted Tree.

Boosted Tree definisce un metodo ensemble che crea una serie di alberi decisionali sequenziali, dove ogni albero dipende dai risultati degli alberi precedenti nel tentativo di ridurre progressivamente l'errore. Si concentra sui pesi degli elementi classificati in modo errato e regola l'adattamento per il classificatore successivo al fine di correggere.

Ci sono diversi modi per adattare questo modello (vedi `help("boost_tree")`). In questo esempio, adatteremo i Boosted Tree tramite il motore `xgboost`.


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

> ✅ Si prega di consultare:
>
> -   [Machine Learning for Social Scientists](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> -   <https://algotech.netlify.app/blog/xgboost/> - Esplora il modello AdaBoost, che rappresenta una buona alternativa a xgboost.
>
> per saperne di più sui classificatori Ensemble.

## 4. Extra - confronto tra modelli multipli

Abbiamo adattato un bel numero di modelli in questo laboratorio 🙌. Può diventare noioso o complicato creare molti workflow da diversi set di preprocessori e/o specifiche di modelli e poi calcolare le metriche di performance una per una.

Vediamo se possiamo affrontare questo problema creando una funzione che adatta una lista di workflow al set di addestramento e poi restituisce le metriche di performance basate sul set di test. Utilizzeremo `map()` e `map_dfr()` dal pacchetto [purrr](https://purrr.tidyverse.org/) per applicare funzioni a ciascun elemento della lista.

> Le funzioni [`map()`](https://purrr.tidyverse.org/reference/map.html) ti permettono di sostituire molti cicli for con codice che è sia più conciso che più facile da leggere. Il miglior posto per imparare le funzioni [`map()`](https://purrr.tidyverse.org/reference/map.html) è il [capitolo sull'iterazione](http://r4ds.had.co.nz/iteration.html) in R for data science.


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


Il pacchetto [**workflowset**](https://workflowsets.tidymodels.org/) permette agli utenti di creare e adattare facilmente un gran numero di modelli, ma è principalmente progettato per lavorare con tecniche di campionamento come la `validazione incrociata`, un approccio che dobbiamo ancora trattare.

## **🚀Sfida**

Ognuna di queste tecniche ha un gran numero di parametri che puoi modificare, ad esempio `cost` negli SVM, `neighbors` in KNN, `mtry` (Predittori Selezionati Casualmente) in Random Forest.

Ricerca i parametri predefiniti di ciascuno e pensa a cosa significherebbe modificare questi parametri per la qualità del modello.

Per saperne di più su un particolare modello e i suoi parametri, usa: `help("model")`, ad esempio `help("rand_forest")`.

> In pratica, di solito *stimiamo* i *valori migliori* per questi parametri allenando molti modelli su un `set di dati simulato` e misurando quanto bene si comportano tutti questi modelli. Questo processo si chiama **ottimizzazione**.

### [**Quiz post-lezione**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **Revisione e Studio Autonomo**

C'è molto gergo in queste lezioni, quindi prenditi un momento per rivedere [questa lista](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott) di terminologia utile!

#### GRAZIE A:

[`Allison Horst`](https://twitter.com/allison_horst/) per aver creato le incredibili illustrazioni che rendono R più accogliente e coinvolgente. Trova altre illustrazioni nella sua [galleria](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

[Cassie Breviu](https://www.twitter.com/cassieview) e [Jen Looper](https://www.twitter.com/jenlooper) per aver creato la versione originale in Python di questo modulo ♥️

Buono studio,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Student Ambassador.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="569"/>
   <figcaption>Illustrazione di @allison_horst</figcaption>



---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale effettuata da un traduttore umano. Non siamo responsabili per eventuali incomprensioni o interpretazioni errate derivanti dall'uso di questa traduzione.
